# Bonus: Temperature Analysis I

In [1]:
import pandas as pd
from datetime import datetime as dt
from scipy.stats import ttest_ind
import numpy as np
from scipy import stats

In [2]:
import numpy as np
import pandas as pd
import datetime as dt

In [3]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func

In [4]:
# create engine to hawaii.sqlite
engine = create_engine("sqlite:///hawaii.sqlite")

In [5]:
# reflect an existing database into a new model
Base = automap_base()
# reflect the tables
Base.prepare(engine, reflect=True)

In [6]:
# View all of the classes that automap found
Base.classes.keys()

['measurement', 'station']

In [7]:
# Save references to each table
Measurement = Base.classes.measurement
Station = Base.classes.station

In [8]:
# "tobs" is "temperature observations"
df = pd.read_csv('hawaii_measurements.csv')

# Show results
df.head()

,station,date,prcp,tobs
0,USC00519397,2010-01-01,0.08,65
1,USC00519397,2010-01-02,0.00,63
2,USC00519397,2010-01-03,0.00,74
3,USC00519397,2010-01-04,0.00,76
4,USC00519397,2010-01-06,NaN,73


In [9]:
# Convert the date column format from string to datetime
df['date'] = pd.to_datetime(df['date'])
df.date = pd.to_datetime(df.date, infer_datetime_format=True)

# Show results
df.head()

,station,date,prcp,tobs
0,USC00519397,2010-01-01,0.08,65
1,USC00519397,2010-01-02,0.00,63
2,USC00519397,2010-01-03,0.00,74
3,USC00519397,2010-01-04,0.00,76
4,USC00519397,2010-01-06,NaN,73


In [10]:
# Set the date column as the DataFrame index
df.set_index(df['date'], drop=False, append=False, inplace=False, verify_integrity=False).drop('date', 1)

,station,prcp,tobs
date,,,
2010-01-01,USC00519397,0.08,65
2010-01-02,USC00519397,0.00,63
2010-01-03,USC00519397,0.00,74
2010-01-04,USC00519397,0.00,76
2010-01-06,USC00519397,NaN,73
...,...,...,...
2017-08-19,USC00516128,0.09,71
2017-08-20,USC00516128,NaN,78
2017-08-21,USC00516128,0.56,76


In [11]:
# Drop the date column
df.reset_index(drop=True, inplace=True)

# Show results
df.head()

,station,date,prcp,tobs
0,USC00519397,2010-01-01,0.08,65
1,USC00519397,2010-01-02,0.00,63
2,USC00519397,2010-01-03,0.00,74
3,USC00519397,2010-01-04,0.00,76
4,USC00519397,2010-01-06,NaN,73


### Compare June and December data across all years 

In [12]:
from scipy import stats

In [13]:
# Create our session (link) from Python to the DB
session = Session(engine)

In [14]:
# Filter data for desired months (June)
june_month = session.query(Measurement.date, Measurement.tobs).filter(func.strftime("%m", Measurement.date)=="06")
june_month

# Show the dataframe
junedf = pd.DataFrame(june_month)

# Show results
junedf

,date,tobs
0,2010-06-01,78.0
1,2010-06-02,76.0
2,2010-06-03,78.0
3,2010-06-04,76.0
4,2010-06-05,77.0
...,...,...
1695,2017-06-26,79.0
1696,2017-06-27,74.0
1697,2017-06-28,74.0
1698,2017-06-29,76.0


In [15]:
# Filter data for desired months (Dec)
dec_month = session.query(Measurement.date, Measurement.tobs).filter(func.strftime("%m", Measurement.date)=="12")
dec_month

# Show the dataframe
decdf = pd.DataFrame(dec_month)

# Show results
decdf

,date,tobs
0,2010-12-01,76.0
1,2010-12-03,74.0
2,2010-12-04,74.0
3,2010-12-06,64.0
4,2010-12-07,64.0
...,...,...
1512,2016-12-27,71.0
1513,2016-12-28,71.0
1514,2016-12-29,69.0
1515,2016-12-30,65.0


In [17]:
# Identify the average temperature for June
# Identify the average temperature for December
average_temp = pd.DataFrame([{"month": "June", "Average Temp": junedf["tobs"].mean()}, 
                             {"month": "Dec", "Average Temp": decdf["tobs"].mean()}])

# Show results
average_temp

,month,Average Temp
0,June,74.944118
1,Dec,71.041529


In [18]:
# Create collections of temperature data
june_temp = junedf["tobs"]
dec_temp = decdf["tobs"]

# Show results
june_temp, dec_temp

(0       78.0
 1       76.0
 2       78.0
 3       76.0
 4       77.0
         ... 
 1695    79.0
 1696    74.0
 1697    74.0
 1698    76.0
 1699    75.0
 Name: tobs, Length: 1700, dtype: float64,
 0       76.0
 1       74.0
 2       74.0
 3       64.0
 4       64.0
         ... 
 1512    71.0
 1513    71.0
 1514    69.0
 1515    65.0
 1516    65.0
 Name: tobs, Length: 1517, dtype: float64)

In [19]:
# Run unpaired t-test
stats.ttest_ind(june_temp, dec_temp)

Ttest_indResult(statistic=31.60372399000329, pvalue=3.9025129038616655e-191)

### Analysis

The p-value is higher than 0.05. It is not statistically significant and indicates strong evidence for the null hypothesis.